In [1]:
import warnings
import tensorflow as tf
import glob
from tqdm import tqdm
import midi_manipulation
import numpy as np
from tensorflow.python.ops import control_flow_ops
from distutils.version import LooseVersion
from utils import *

data_path = './data/BigPackMIDI_EDM/block1/'
num_songs = 165
seq_length = 60

input_songs, target_songs = get_song_matrixes(data_path, num_songs, seq_length)

# Vectorizing songs with shape (num_seqs, seq_length, 156)
input_songs = np.array(input_songs)
target_songs = np.array(target_songs) # TODO: Maybe remove?

# Finding and saving all the tokens in the songs
# TODO: Later change this generating all the possible 156*156 tokens
tokens = get_tokens(input_songs)
num_encoder_tokens = np.array(tokens).shape[0]
num_decoder_tokens = np.array(tokens).shape[0]


print('[*] Embedding Songs')
# Tokenizings the songs from shape (seq_length, 156) to (seq_length, num_encoder_tokens)
embeded_input_songs = get_embeded_songs(input_songs, tokens, num_encoder_tokens)
embeded_target_songs = embeded_input_songs

[*] Converting songs to matrix


100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:49<00:00,  3.33it/s]


[*] Converted 163 songs to matrix



(60, 156)
Processing song: 0/1441
Processing song: 50/1441
Processing song: 100/1441
Processing song: 150/1441
Processing song: 200/1441
Processing song: 250/1441
Processing song: 300/1441
Processing song: 350/1441
Processing song: 400/1441
Processing song: 450/1441
Processing song: 500/1441
Processing song: 550/1441
Processing song: 600/1441
Processing song: 650/1441
Processing song: 700/1441
Processing song: 750/1441
Processing song: 800/1441
Processing song: 850/1441
Processing song: 900/1441
Processing song: 950/1441
Processing song: 1000/1441
Processing song: 1050/1441
Processing song: 1100/1441
Processing song: 1150/1441
Processing song: 1200/1441
Processing song: 1250/1441
Processing song: 1300/1441
Processing song: 1350/1441
Processing song: 1400/1441
[*] Embedding Songs
Processing embed: 0/1441
Processing embed: 50/1441
Processing embed: 100/1441
Processing embed: 150/1441
Processing embed: 200/1441
Processing embed: 250/1441
Processing emb

In [2]:
print(np.array(embeded_input_songs[0]).shape)
print(np.array(embed_song_to_song(embeded_input_songs[0], tokens)).shape)

# Finding the longest song in the dataset
max_encoder_seq_length = max([len(song) for song in embeded_input_songs])
max_decoder_seq_length = max([len(song) for song in embeded_target_songs])

# Data report
print('Number of samples:', len(embeded_input_songs))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# Get input data in shape (num_sample, max_seq_length, num_tokens)
encoder_input_data, decoder_input_data, decoder_target_data = get_input_data(
                                                                        embeded_input_songs, 
                                                                        embeded_target_songs,
                                                                        max_encoder_seq_length, 
                                                                        num_encoder_tokens, 
                                                                        max_decoder_seq_length, 
                                                                        num_decoder_tokens)

(60, 9483)
(60, 156)
Number of samples: 1441
Number of unique input tokens: 9483
Number of unique output tokens: 9483
Max sequence length for inputs: 60
Max sequence length for outputs: 60
(1441, 60, 9483) (1441, 60, 9483)

Encoder input data shape: (1441, 60, 9483)
Decoder input data shape: (1441, 60, 9483)
Decoder target data shape: (1441, 60, 9483)



In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Hyperparameters
batch_size  =   32       # Batch size for training.
epochs      =   70       # Number of epochs to train for.
latent_dim  =   1024     # Latent dimensionality of the encoding space.
training = False
# TODO: Add all the other hyperparameters
#num_samples =   10000   # Number of samples to train on.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print()
print(model.summary())
print()

Using TensorFlow backend.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 9483)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 9483)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 1024), 43040768    input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 1024), 43040768    input_2[0][0]                    
                                                                 lstm_1[0][1]                     
         

In [4]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy')

training = True

print('[*] Starting Training')
if training:
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              #validation_split=0.1 - TODO: fix overfitting or not?
             )
else:
    model.load_weights('s2s'+'edm'+'.h5')
print('[*] Ready to be used \n\n')

[*] Starting Training
Epoch 1/70
1441/1441 [==============================] - ETA: 6:04 - loss: 9.309 - ETA: 3:28 - loss: 9.290 - ETA: 2:37 - loss: 9.255 - ETA: 2:09 - loss: 8.783 - ETA: 2:01 - loss: 8.446 - ETA: 1:48 - loss: 8.012 - ETA: 1:38 - loss: 7.659 - ETA: 1:32 - loss: 7.555 - ETA: 1:26 - loss: 7.375 - ETA: 1:20 - loss: 7.223 - ETA: 1:18 - loss: 7.146 - ETA: 1:14 - loss: 7.129 - ETA: 1:09 - loss: 7.118 - ETA: 1:06 - loss: 7.048 - ETA: 1:04 - loss: 7.075 - ETA: 1:03 - loss: 7.076 - ETA: 1:00 - loss: 6.976 - ETA: 57s - loss: 6.938 - ETA: 55s - loss: 6.91 - ETA: 52s - loss: 6.90 - ETA: 49s - loss: 6.88 - ETA: 47s - loss: 6.84 - ETA: 44s - loss: 6.87 - ETA: 41s - loss: 6.84 - ETA: 39s - loss: 6.83 - ETA: 37s - loss: 6.83 - ETA: 34s - loss: 6.80 - ETA: 32s - loss: 6.77 - ETA: 30s - loss: 6.81 - ETA: 28s - loss: 6.75 - ETA: 26s - loss: 6.71 - ETA: 24s - loss: 6.72 - ETA: 22s - loss: 6.67 - ETA: 20s - loss: 6.65 - ETA: 18s - loss: 6.64 - ETA: 16s - loss: 6.66 - ETA: 14s - loss: 6.65 -

1441/1441 [==============================] - ETA: 5:13 - loss: 4.993 - ETA: 5:10 - loss: 5.259 - ETA: 4:52 - loss: 5.141 - ETA: 4:41 - loss: 4.877 - ETA: 4:07 - loss: 4.776 - ETA: 4:10 - loss: 4.861 - ETA: 4:06 - loss: 4.823 - ETA: 4:04 - loss: 4.777 - ETA: 3:58 - loss: 4.686 - ETA: 3:55 - loss: 4.691 - ETA: 3:50 - loss: 4.739 - ETA: 3:45 - loss: 4.752 - ETA: 3:30 - loss: 4.793 - ETA: 3:25 - loss: 4.778 - ETA: 3:12 - loss: 4.799 - ETA: 3:07 - loss: 4.809 - ETA: 2:57 - loss: 4.760 - ETA: 2:52 - loss: 4.812 - ETA: 2:47 - loss: 4.776 - ETA: 2:41 - loss: 4.796 - ETA: 2:34 - loss: 4.782 - ETA: 2:26 - loss: 4.738 - ETA: 2:18 - loss: 4.756 - ETA: 2:10 - loss: 4.748 - ETA: 2:02 - loss: 4.758 - ETA: 1:56 - loss: 4.740 - ETA: 1:50 - loss: 4.755 - ETA: 1:43 - loss: 4.772 - ETA: 1:37 - loss: 4.763 - ETA: 1:31 - loss: 4.805 - ETA: 1:25 - loss: 4.855 - ETA: 1:18 - loss: 4.863 - ETA: 1:12 - loss: 4.865 - ETA: 1:06 - loss: 4.853 - ETA: 1:00 - loss: 4.877 - ETA: 54s - loss: 4.875 - ETA: 48s - loss: 4.8

1441/1441 [==============================] - ETA: 5:04 - loss: 3.379 - ETA: 5:05 - loss: 3.267 - ETA: 4:09 - loss: 3.109 - ETA: 4:05 - loss: 3.208 - ETA: 4:04 - loss: 3.121 - ETA: 3:37 - loss: 3.058 - ETA: 3:35 - loss: 3.142 - ETA: 3:37 - loss: 3.298 - ETA: 3:37 - loss: 3.309 - ETA: 3:34 - loss: 3.235 - ETA: 3:30 - loss: 3.158 - ETA: 3:26 - loss: 3.160 - ETA: 3:19 - loss: 3.136 - ETA: 3:12 - loss: 3.174 - ETA: 3:06 - loss: 3.164 - ETA: 3:01 - loss: 3.071 - ETA: 2:54 - loss: 3.079 - ETA: 2:49 - loss: 3.087 - ETA: 2:40 - loss: 3.110 - ETA: 2:34 - loss: 3.103 - ETA: 2:28 - loss: 3.152 - ETA: 2:20 - loss: 3.152 - ETA: 2:15 - loss: 3.126 - ETA: 2:10 - loss: 3.136 - ETA: 2:01 - loss: 3.126 - ETA: 1:55 - loss: 3.099 - ETA: 1:49 - loss: 3.087 - ETA: 1:44 - loss: 3.124 - ETA: 1:38 - loss: 3.083 - ETA: 1:32 - loss: 3.076 - ETA: 1:25 - loss: 3.098 - ETA: 1:19 - loss: 3.073 - ETA: 1:13 - loss: 3.073 - ETA: 1:07 - loss: 3.078 - ETA: 1:01 - loss: 3.076 - ETA: 55s - loss: 3.078 - ETA: 49s - loss: 3.0

1441/1441 [==============================] - ETA: 2:02 - loss: 1.360 - ETA: 3:47 - loss: 1.417 - ETA: 4:06 - loss: 1.317 - ETA: 4:16 - loss: 1.340 - ETA: 4:17 - loss: 1.427 - ETA: 4:15 - loss: 1.408 - ETA: 3:49 - loss: 1.422 - ETA: 3:47 - loss: 1.411 - ETA: 3:46 - loss: 1.426 - ETA: 3:45 - loss: 1.486 - ETA: 3:31 - loss: 1.495 - ETA: 3:29 - loss: 1.505 - ETA: 3:17 - loss: 1.501 - ETA: 3:12 - loss: 1.503 - ETA: 3:07 - loss: 1.524 - ETA: 3:02 - loss: 1.507 - ETA: 2:57 - loss: 1.492 - ETA: 2:53 - loss: 1.481 - ETA: 2:48 - loss: 1.485 - ETA: 2:43 - loss: 1.503 - ETA: 2:34 - loss: 1.514 - ETA: 2:29 - loss: 1.509 - ETA: 2:23 - loss: 1.506 - ETA: 2:17 - loss: 1.515 - ETA: 2:08 - loss: 1.514 - ETA: 2:02 - loss: 1.517 - ETA: 1:53 - loss: 1.515 - ETA: 1:47 - loss: 1.501 - ETA: 1:39 - loss: 1.510 - ETA: 1:34 - loss: 1.497 - ETA: 1:28 - loss: 1.489 - ETA: 1:23 - loss: 1.488 - ETA: 1:15 - loss: 1.478 - ETA: 1:09 - loss: 1.483 - ETA: 1:03 - loss: 1.473 - ETA: 57s - loss: 1.486 - ETA: 51s - loss: 1.4

1441/1441 [==============================] - ETA: 5:26 - loss: 0.653 - ETA: 3:43 - loss: 0.543 - ETA: 4:06 - loss: 0.650 - ETA: 4:16 - loss: 0.641 - ETA: 4:01 - loss: 0.604 - ETA: 4:03 - loss: 0.618 - ETA: 4:06 - loss: 0.606 - ETA: 4:03 - loss: 0.614 - ETA: 4:01 - loss: 0.635 - ETA: 3:57 - loss: 0.634 - ETA: 3:49 - loss: 0.637 - ETA: 3:44 - loss: 0.625 - ETA: 3:35 - loss: 0.639 - ETA: 3:27 - loss: 0.639 - ETA: 3:22 - loss: 0.649 - ETA: 3:13 - loss: 0.646 - ETA: 3:08 - loss: 0.651 - ETA: 2:58 - loss: 0.647 - ETA: 2:51 - loss: 0.653 - ETA: 2:45 - loss: 0.652 - ETA: 2:35 - loss: 0.642 - ETA: 2:29 - loss: 0.644 - ETA: 2:23 - loss: 0.650 - ETA: 2:17 - loss: 0.644 - ETA: 2:10 - loss: 0.644 - ETA: 2:04 - loss: 0.636 - ETA: 1:58 - loss: 0.636 - ETA: 1:52 - loss: 0.638 - ETA: 1:47 - loss: 0.635 - ETA: 1:42 - loss: 0.632 - ETA: 1:33 - loss: 0.633 - ETA: 1:26 - loss: 0.628 - ETA: 1:20 - loss: 0.627 - ETA: 1:14 - loss: 0.624 - ETA: 1:06 - loss: 0.627 - ETA: 59s - loss: 0.626 - ETA: 53s - loss: 0.6

...

1441/1441 [==============================] - ETA: 5:24 - loss: 0.127 - ETA: 5:23 - loss: 0.108 - ETA: 5:06 - loss: 0.106 - ETA: 4:57 - loss: 0.106 - ETA: 4:46 - loss: 0.110 - ETA: 4:40 - loss: 0.113 - ETA: 4:31 - loss: 0.113 - ETA: 4:07 - loss: 0.114 - ETA: 4:02 - loss: 0.112 - ETA: 3:58 - loss: 0.110 - ETA: 3:54 - loss: 0.108 - ETA: 3:38 - loss: 0.109 - ETA: 3:33 - loss: 0.109 - ETA: 3:27 - loss: 0.111 - ETA: 3:21 - loss: 0.111 - ETA: 3:08 - loss: 0.110 - ETA: 3:02 - loss: 0.111 - ETA: 2:57 - loss: 0.111 - ETA: 2:51 - loss: 0.112 - ETA: 2:46 - loss: 0.111 - ETA: 2:35 - loss: 0.110 - ETA: 2:30 - loss: 0.111 - ETA: 2:24 - loss: 0.110 - ETA: 2:18 - loss: 0.109 - ETA: 2:12 - loss: 0.110 - ETA: 2:07 - loss: 0.109 - ETA: 2:00 - loss: 0.109 - ETA: 1:54 - loss: 0.109 - ETA: 1:48 - loss: 0.108 - ETA: 1:41 - loss: 0.108 - ETA: 1:35 - loss: 0.108 - ETA: 1:26 - loss: 0.108 - ETA: 1:20 - loss: 0.108 - ETA: 1:14 - loss: 0.108 - ETA: 1:07 - loss: 0.108 - ETA: 1:01 - loss: 0.109 - ETA: 54s - loss: 0.

1441/1441 [==============================] - ETA: 5:02 - loss: 0.084 - ETA: 4:59 - loss: 0.073 - ETA: 4:14 - loss: 0.074 - ETA: 4:16 - loss: 0.078 - ETA: 4:15 - loss: 0.077 - ETA: 4:17 - loss: 0.078 - ETA: 4:02 - loss: 0.078 - ETA: 4:00 - loss: 0.079 - ETA: 3:58 - loss: 0.079 - ETA: 3:38 - loss: 0.079 - ETA: 3:30 - loss: 0.079 - ETA: 3:24 - loss: 0.078 - ETA: 3:22 - loss: 0.078 - ETA: 3:16 - loss: 0.078 - ETA: 3:11 - loss: 0.078 - ETA: 2:59 - loss: 0.079 - ETA: 2:54 - loss: 0.079 - ETA: 2:49 - loss: 0.079 - ETA: 2:44 - loss: 0.079 - ETA: 2:39 - loss: 0.080 - ETA: 2:30 - loss: 0.080 - ETA: 2:24 - loss: 0.080 - ETA: 2:19 - loss: 0.080 - ETA: 2:13 - loss: 0.080 - ETA: 2:07 - loss: 0.080 - ETA: 2:02 - loss: 0.080 - ETA: 1:56 - loss: 0.080 - ETA: 1:49 - loss: 0.079 - ETA: 1:43 - loss: 0.079 - ETA: 1:37 - loss: 0.079 - ETA: 1:31 - loss: 0.079 - ETA: 1:25 - loss: 0.079 - ETA: 1:19 - loss: 0.079 - ETA: 1:12 - loss: 0.079 - ETA: 1:06 - loss: 0.079 - ETA: 59s - loss: 0.079 - ETA: 52s - loss: 0.0

1441/1441 [==============================] - ETA: 5:27 - loss: 0.063 - ETA: 4:00 - loss: 0.065 - ETA: 4:15 - loss: 0.060 - ETA: 4:20 - loss: 0.061 - ETA: 4:19 - loss: 0.060 - ETA: 3:51 - loss: 0.061 - ETA: 3:52 - loss: 0.060 - ETA: 3:51 - loss: 0.059 - ETA: 3:37 - loss: 0.059 - ETA: 3:34 - loss: 0.060 - ETA: 3:23 - loss: 0.060 - ETA: 3:20 - loss: 0.060 - ETA: 3:17 - loss: 0.060 - ETA: 3:12 - loss: 0.061 - ETA: 3:06 - loss: 0.061 - ETA: 3:03 - loss: 0.060 - ETA: 2:51 - loss: 0.061 - ETA: 2:47 - loss: 0.060 - ETA: 2:43 - loss: 0.060 - ETA: 2:37 - loss: 0.060 - ETA: 2:32 - loss: 0.060 - ETA: 2:27 - loss: 0.060 - ETA: 2:22 - loss: 0.060 - ETA: 2:17 - loss: 0.061 - ETA: 2:08 - loss: 0.061 - ETA: 2:02 - loss: 0.061 - ETA: 1:56 - loss: 0.061 - ETA: 1:50 - loss: 0.060 - ETA: 1:44 - loss: 0.060 - ETA: 1:38 - loss: 0.061 - ETA: 1:31 - loss: 0.060 - ETA: 1:25 - loss: 0.060 - ETA: 1:19 - loss: 0.061 - ETA: 1:12 - loss: 0.061 - ETA: 1:06 - loss: 0.061 - ETA: 1:00 - loss: 0.061 - ETA: 53s - loss: 0.

1441/1441 [==============================] - ETA: 4:41 - loss: 0.038 - ETA: 5:02 - loss: 0.039 - ETA: 4:31 - loss: 0.041 - ETA: 4:33 - loss: 0.041 - ETA: 4:21 - loss: 0.042 - ETA: 4:19 - loss: 0.042 - ETA: 4:20 - loss: 0.044 - ETA: 4:09 - loss: 0.044 - ETA: 4:04 - loss: 0.044 - ETA: 3:43 - loss: 0.044 - ETA: 3:40 - loss: 0.044 - ETA: 3:36 - loss: 0.044 - ETA: 3:32 - loss: 0.044 - ETA: 3:16 - loss: 0.044 - ETA: 3:12 - loss: 0.044 - ETA: 3:07 - loss: 0.044 - ETA: 3:02 - loss: 0.044 - ETA: 2:57 - loss: 0.045 - ETA: 2:52 - loss: 0.045 - ETA: 2:46 - loss: 0.045 - ETA: 2:40 - loss: 0.045 - ETA: 2:33 - loss: 0.044 - ETA: 2:28 - loss: 0.044 - ETA: 2:22 - loss: 0.044 - ETA: 2:15 - loss: 0.044 - ETA: 2:09 - loss: 0.044 - ETA: 2:03 - loss: 0.044 - ETA: 1:57 - loss: 0.044 - ETA: 1:50 - loss: 0.044 - ETA: 1:43 - loss: 0.044 - ETA: 1:37 - loss: 0.044 - ETA: 1:28 - loss: 0.044 - ETA: 1:22 - loss: 0.044 - ETA: 1:14 - loss: 0.044 - ETA: 1:07 - loss: 0.044 - ETA: 1:01 - loss: 0.044 - ETA: 54s - loss: 0.

1441/1441 [==============================] - ETA: 4:06 - loss: 0.037 - ETA: 4:33 - loss: 0.038 - ETA: 4:44 - loss: 0.038 - ETA: 4:46 - loss: 0.037 - ETA: 4:45 - loss: 0.038 - ETA: 4:40 - loss: 0.038 - ETA: 4:31 - loss: 0.037 - ETA: 4:27 - loss: 0.037 - ETA: 4:16 - loss: 0.037 - ETA: 4:08 - loss: 0.038 - ETA: 4:03 - loss: 0.038 - ETA: 3:54 - loss: 0.038 - ETA: 3:47 - loss: 0.039 - ETA: 3:41 - loss: 0.039 - ETA: 3:26 - loss: 0.039 - ETA: 3:20 - loss: 0.039 - ETA: 3:14 - loss: 0.039 - ETA: 3:08 - loss: 0.039 - ETA: 3:02 - loss: 0.039 - ETA: 2:54 - loss: 0.039 - ETA: 2:47 - loss: 0.038 - ETA: 2:40 - loss: 0.038 - ETA: 2:34 - loss: 0.038 - ETA: 2:27 - loss: 0.038 - ETA: 2:21 - loss: 0.038 - ETA: 2:14 - loss: 0.038 - ETA: 2:06 - loss: 0.038 - ETA: 2:00 - loss: 0.038 - ETA: 1:53 - loss: 0.038 - ETA: 1:46 - loss: 0.038 - ETA: 1:39 - loss: 0.038 - ETA: 1:31 - loss: 0.038 - ETA: 1:24 - loss: 0.038 - ETA: 1:17 - loss: 0.038 - ETA: 1:10 - loss: 0.038 - ETA: 1:03 - loss: 0.038 - ETA: 56s - loss: 0.

In [5]:
# Save model
model.save('s2s'+'edm'+'.h5')

C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 1024) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [72]:
# song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/BigPackMIDI_EDM/block1/' + song_name))

# print(song.shape)
# midi_manipulation.noteStateMatrixToMidi(song[100:200], 'test'+song_name)

C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\midi-0.2.3-py3.6.egg\midi\fileio.py:68: Warning: Unknown Meta MIDI Event: 9
  warn("Unknown Meta MIDI Event: " + repr(cmd), Warning)
C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\midi-0.2.3-py3.6.egg\midi\fileio.py:68: Warning: Unknown Meta MIDI Event: 10
  warn("Unknown Meta MIDI Event: " + repr(cmd), Warning)


(1969, 156)


In [6]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [8]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print(np.array(states_value).shape)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]

    return song_matrix


seq_length = max_encoder_seq_length
song_name = 'Idina-Menzel-Let-It-Go-Anonymous-20140127194031-nonstop2k.com' + '.mid'
song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/BigPackMIDI_EDM/block1/' + song_name))


encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
print(np.array(encoder_input_data).shape)        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)

# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq = encoder_input_data1[:-1]

decoded_songs = []
decoded_songs.append(decode_sequence(np.array(encoder_input_data1[:-1])))
for i in range(1):
    decoded_songs.append(decode_sequence((
                    np.reshape(decoded_songs[-1], 
                        (1, decoded_songs[-1].shape[0], decoded_songs[-1].shape[1])))))


decoded_song = embed_song_to_song(np.concatenate(decoded_songs, axis = 0), tokens)

# Converting Song to midi from matrix
print('[*] Converting and saving song')
midi_manipulation.noteStateMatrixToMidi(decoded_song, 'edm_'+song_name)

C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\midi-0.2.3-py3.6.egg\midi\fileio.py:68: Warning: Unknown Meta MIDI Event: 9
C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\midi-0.2.3-py3.6.egg\midi\fileio.py:68: Warning: Unknown Meta MIDI Event: 10


(32, 60, 156)
Processing embed: 0/32
[*] Encoding-Decoding
(2, 31, 1024)
[iter:0] [max_decoder_seq_length: 60]
(2, 1, 1024)
[iter:0] [max_decoder_seq_length: 60]
[*] Converting and saving song


In [67]:
midi_manipulation.noteStateMatrixToMidi(decoded_song, 'edm_'+song_name)

In [15]:
np.sum(decoded_song)

2937

In [134]:
def mixing_sequence(input_seq1, input_seq2, ratio = (2, 2)):
    # Encode the input as state vectors.
    states_value2 = encoder_model.predict(input_seq1)
    states_value = encoder_model.predict(input_seq2)
    
    for i, states1 in enumerate(states_value):
        for j, states2 in enumerate(states1):
            for k, states3 in enumerate(states2):
                states_value[i][j][k] = (states3*ratio[0] + states_value2[i][j][k]*ratio[1])/2
        
            
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]


    return song_matrix

seq_length = max_encoder_seq_length

song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/edm_essential_melodies/Ralph Cowell EDM Essential Melodies MIDI Vol. 1 - 6.mid'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)

# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq1 = encoder_input_data1[:-1]


song = np.array(midi_manipulation.midiToNoteStateMatrix('./data/edm_essential_melodies/Ralph Cowell EDM Essential Melodies MIDI Vol. 2 - 10.mid'))

encoder_input_data = []
if np.array(song).shape[0] > 50:   
    length = np.array(song).shape[0]
    for j in range(length // seq_length):
        encoder_input_data.append(song[seq_length*j:seq_length*(j+1)])
        
        
#encoder_input_data = get_embeded_songs(encoder_input_data, tokens, num_encoder_tokens)                                                       
encoder_input_data1 = []
for i, song in enumerate(encoder_input_data):
    if(i%50==0):
        print('Processing embed: {}/{}'.format(i,np.array(encoder_input_data).shape[0]))
    embed_song = []
    for i, state in enumerate(song):
        idx = state_to_token(state, tokens)
        embed = np.zeros(num_encoder_tokens)
        embed[idx] = 1
        embed_song.append(embed)
    encoder_input_data1.append(embed_song)
    
# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq2 = encoder_input_data1[:-1]

decoded_songs = []
decoded_songs.append(mixing_sequence(input_seq1, input_seq2))
for i in range(2):
    decoded_songs.append(decode_sequence((
                    np.reshape(decoded_songs[-1], 
                        (1, decoded_songs[-1].shape[0], decoded_songs[-1].shape[1])))))


decoded_song = embed_song_to_song(np.concatenate(decoded_songs, axis = 0), tokens)
edm+=1
# Converting Song to midi from matrix
print('[*] Converting and saving song')
midi_manipulation.noteStateMatrixToMidi(decoded_song, 'mixing_edm'+str(edm))

Processing embed: 0/3
[*] Encoding-Decoding
Processing embed: 0/3
[*] Encoding-Decoding
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
(2, 1, 512)
[iter:0] [max_decoder_seq_length: 20]
[*] Converting and saving song


In [ ]:
np.zeros((1,1,1,11432)) + np.zeros((2,1,1024)).tolist()